# Session 2.4: Document Processing

## Quellen

- Docling: https://github.com/docling-project/docling
    - als API lokal zur Verfügung stellen oder als Library in Python importieren
    

## Praxis

### 🛠️ Imports

In [1]:
from aiworkshop_utils.standardlib_imports import os, json, base64, logging, Optional, Any, Dict, List, pprint, shutil, glob
from aiworkshop_utils.thirdparty_imports import AutoTokenizer, load_dotenv, requests, BaseModel, Field, pd, cosine_similarity, plt, np, DataType, MilvusClient, tqdm, SentenceTransformer
from aiworkshop_utils.custom_utils import show_pretty_json, encode_image, load_md_sections_from_path
from aiworkshop_utils.jupyter_imports import Markdown, HTML, widgets, display, JSON
from aiworkshop_utils.docling_imports import PyPdfiumDocumentBackend, HybridChunker, InputFormat, AcceleratorDevice, AcceleratorOptions, PdfPipelineOptions, DocumentConverter, PdfFormatOption, WordFormatOption, SimplePipeline
from aiworkshop_utils.openai_imports import openaisdk_client_chat_completions_create
from aiworkshop_utils import config
from aiworkshop_utils import embedders

### 💡 Konzept: Document Processing - das Fundament eines erfolgreichen RAG-Systems

- Was ist Document Processing?
    - Unstrukturierte/Semi-Strukturierte Daten (PDFs, Word-Dateien, Webseiten, Scans, E-Mails, Powerpoint, Excel, MD ...) werden so aufbereitet, dass die Inhalte effizient durchsucht werden können
    - Die Dokumente können dann auch als Grundlage für einen RAG-Prozess (Retrieval Augmented Generation) dienen

- Der ganze RAG-Prozess -> Welche Schritte sind hier inkludiert?
    - Parsing: Dokumente analysieren und in maschinenlesbare Textdaten überführen (z.B. Markdown oder JSON)
    - Chunking: Textdaten werden in sinnvolle, inhaltlich zusammenhängende Abschnitte = Chunks zerlegt, um für semantische Suche zu dienen
    - Embedding: Chunks werden in Vektorraum eines Embedding-Modelles eingebettet
    - Indexing: Diese eingebetteten Chunks = Vektoren werden gemeinsam mit den Metadaten (richtiger Text, Überschriften, Seitennummer, ...) in einer Vektordatenbank indexiert
    - Retrieval: Ein Prompt eines Users an ein Generations-Modell wird mit dem gleichen Embedding-Modell eingebettet -> daraus entstehen Vektoren. Damit werden die ähnlichsten Vektoren der eingebetteten Chunks gesucht und gereiht (z.B. die besten 3)
    - Generation: Die Texte aus den Metadaten der Ergebnisse des Retrievals werden in den Kontext der Konversation eines Users mit einem Generations-Modell gebracht und im Systemprompt wird das Modell angewiesen, sich bei der Beantwortung der Frage auf diese Texte zu beziehen.
    - -> darum heißt es Retrieval-Augmented-Generation: Es wird vom Generations-Modell ein Text generiert, der durch Retrieval bereichert ("erweitert") wurde

- Welche Technologien werden beim Document Processing angewandt?
    - Textextraktion, z.B. durch OCR (Optical Character Recognition)
    - Bereinigung/Vorverarbeitung (Fußzeilen, Formatierungen usw. entfernen)
    - Chunking: Unterteilung der Texte in sinnvolle Abschnitte - typischerweise 100-500 Wörter - wichtig ist, dass Kontext erhalten bleibt in einem Chunk!
    - Embedding -> Jeder Chunk wird in einen Vektor mit seinen Dimensionen umgewandelt
        - z.B. "Document Processing ist wichtig für den RAG Prozess." -> wenn ein Modell mit 4 Dimensionen einbettet, z.B. Vektor: [-0.3402, 1.2039, 2.2039, -1.1230]
    - Speicherung in einer Vektordatenbank: Embeddings und Metadaten (Quelle, Abschnitt, Seitenzahl, Topic, ...)

- Was passiert, wenn es kein gutes Document Processing gibt?
    - Relevante Informationen werden nicht gefunden (weil z.B. zu große Chunks)
    - Es werden falsche oder unvollständige Antworten gegeben (unzureichende oder falsch struktuierte Kontextinformationen)
    - semantischer Kontext kannn verloren gehen (wenn z.B. Chunk-Grenzen willlkürlich gesetzt sind oder inhaltlich logisch zerhackt werden)

- Welche gute Software gibt es dafür?
    - z.B. Docling (von IBM, aber Open Source)
    - Docling als Service lokal mit API ansprechbar hosten oder als Modul in Python importieren (so wie wir es machen)


### ⚡ Ein Dokument mit dem Default Docling DocumentConverter konvertieren

![Docling DocumentConverter Architecture](assets/image08_docling-architecture.png)

- DocumentConverter
    - https://docling-project.github.io/docling/reference/document_converter/
- ConversionResult
    - converter.convert(source) ergibt die Klasse ConversionResult
    - https://docling-project.github.io/docling/reference/document_converter/#docling.document_converter.ConversionResult
- export_to_markdown
    - damit kann man das ConversionResult.document zu Markdown exportieren, das man später mit einem Modell embedden kann
    - https://docling-project.github.io/docling/reference/docling_document/#docling_core.types.doc.DoclingDocument.export_to_markdown

In [ ]:
source = "https://arxiv.org/pdf/2408.09869"  # Dokument als lokaler Pfad oder URL
converter = DocumentConverter()
result = converter.convert(source)
print(result.document.export_to_markdown())  # Output: "## Docling Technical Report[...]"

- Optionen, besonders für PDF-Processing, im DocumentConverter einstellen

```
pipeline_options = PdfPipelineOptions()

pipeline_options.do_ocr = True
pipeline_options.do_table_structure = True
pipeline_options.table_structure_options.do_cell_matching = True
pipeline_options.ocr_options.lang = ["en"]
pipeline_options.accelerator_options = AcceleratorOptions(
    num_threads=4,
    device=AcceleratorDevice.CPU
)

converter = DocumentConverter(
    
    allowed_formats=[
        InputFormat.PDF,
        InputFormat.IMAGE,
        InputFormat.DOCX,
        InputFormat.HTML,
        InputFormat.PPTX,
    ],
    
    format_options={
        
        InputFormat.PDF: PdfFormatOption(
            pipeline_options=pipeline_options,
            backend=PyPdfiumDocumentBackend     # OpenSource Google C++-Lib zum Rendern und Verarbeiten von PDF-Dateien
        ),
        
        InputFormat.DOCX: WordFormatOption(
            pipeline_cls=SimplePipeline
        ),
    },
)
```

- Benutzt Docling KI-Modelle im Hintergrund bei der Prozessierung?
    - Ja, z.B. DocLayNet für Layout-Analyse, z.B. TableFormer für die Erkennung von Tabellenstrukturen

### 💡 Konzept: Chunking ("Zerstückelung")

- Geparste Textdaten werden intelligent in Abschnitte aufgeteilt, die jeweils semantisch noch verständlich sind
- Docling Chunking Eintrag: https://docling-project.github.io/docling/concepts/chunking/

In [ ]:
# Chunking durchführen
chunker = HybridChunker(tokenizer="sentence-transformers/all-MiniLM-L6-v2")
chunks = list(chunker.chunk(result.document))

# Ausgabe der Chunks
print(f"\nEs wurden {len(chunks)} Chunks erstellt:\n")
for i, chunk in enumerate(chunks[:5], 1):
    print(f"--- Chunk {i} ---")
    print("Text:", chunk.text.strip()[:300], "...\n")

    meta = getattr(chunk, "meta", None) # meta ist ein Pydantic Model - Docling verwendet Pydantic intern
    headings = getattr(meta, "headings", []) if meta else []
    page_info = getattr(meta, "page_info", "Unknown") if meta else "Unknown"
    content_type = getattr(meta, "content_type", "Unknown") if meta else "Unknown"

    print("Headings:", headings)
    print("Page:", page_info)
    print("Content-Type:", content_type)
    print("-" * 50)

### 🏋️ **[ÜBUNG_2.4.01]** Document-Processing-Klasse erstellen

- *Diese Übung ist Teil von ÜBUNG4. Speichere deine Ergebnisse und Notizen in einem File (Word), füge dann noch die restlichen Übungen dieser Session 2.4 hinzu und lade sie auf Moodle unter "Abgabe Übung 4" bis zum 20.05.25 hoch.*

- Was soll sie können?
    - init (Constructor)
    - Funktion für einen Docling DocumentConverter erstellen mit bestimmten Optionen
    - Funktion für einen Index erstellen für temporären Speicher (noch keine DB)
    - Funktion für Dokument prozessieren
    - Funktion für Chunking des Ergebnisses der Prozessierung (kann gerne ein Chunker von Docling sein)

- darf nicht exakt gleich sein wie die Lösung hier!

- Falls noch Zeit
    - Funktion für Kontext formatieren
    - Funktion für einfache Vektorsuche
    - Funktion für Query

### 🔓 **[LÖSUNG_2.4.01]** Document-Processing-Klasse erstellen

### ⚡ Dokument mit Document Processor Klasse prozessieren


In [3]:
# Example instantiation and usage
ects_guide_processor = DocumentProcessor(embedders.OllamaEmbedder(
    url=config.OAPI_EMBED_URL,
    model_name=config.OMODEL_NOMIC
))

In [4]:
# Update the path to your document
pdf_path = "doc_storage/ECTS_Guide_BSWE-pages-1-10.pdf"

# Check if the document exists
if os.path.exists(pdf_path):
    ects_guide_processor.process_document(pdf_path)
else:
    print(f"Document not found: {pdf_path}")
    print("Please update the pdf_path variable with the correct path to your document.")

Converting document: doc_storage/ECTS_Guide_BSWE-pages-1-10.pdf
Chunking document...


Token indices sequence length is longer than the specified maximum sequence length for this model (596 > 512). Running this sequence through the model will result in indexing errors


Created 18 chunks
Processing chunks and extracting metadata...


Extracting metadata: 100%|██████████| 18/18 [00:00<00:00, 128615.80it/s]


Generating embeddings and building index...
Finished processing document. Index contains 18 chunks.


- Chunk-Schema:

```
{
    "text": "LV Art Course Type, I0859GDI01 = Integrierte Lehrveranstaltung Integrated Course. ...",
    "headings": ["Grundlagen der Informatik", "Foundations of Computer Science"],
    "page_info": 3,
    "content_type": "text",  # oder z. B. "table" oder "figure"
    "vector": [0.123, -0.456, ..., 0.015]  # → 768-dimensionaler Embedding-Vektor vom Modell
}
```

In [7]:
print(len(ects_guide_processor.index))


18


In [13]:
show_pretty_json(ects_guide_processor.index[17])

```json
{
  "text": "LV Nummer Course number, 1 = I0859GPR03. LV Art Course Type, 1 = Integrierte Lehrveranstaltung Integrated Course. Semester, 1 = 3. Lehreinheiten Teaching units, 1 = 60. ECTS, 1 = 6 ECTS. Bewertungsmethode Evaluation method, 1 = Immanenter Pr\u00fcfungscharakter Continuous assessment. Lehrveranstaltungsinhalte Content, 1 = \u2022  Grundlagen des User Interface Designs \u2022  Kriterien f\u00fcr UI - Designs \u2022  Werkzeuge f\u00fcr die Oberfl\u00e4chengestaltung \u2022  Basics of user interface design \u2022  Criteria for UI designs \u2022  Tools for UI design\nLE0435_I_0859_ECTS_Guide_BSWE_2025 KP01LE \u2013 LE0400 erstellt: 29.04.2022/frn",
  "headings": [
    "Human Interface Design"
  ],
  "page_info": null,
  "content_type": null,
  "vector": [
    -0.023945624,
    -0.019180695,
    -0.13686816,
    -0.08257686,
    0.004624458,
    0.049778257,
    0.059064027,
    -0.032191373,
    0.027657645,
    0.0060658427,
    "... (758 more elements)"
  ]
}
```

In [16]:
show_pretty_json(ects_guide_processor.document_info)

```json
{
  "path": "doc_storage/ECTS_Guide_BSWE-pages-1-10.pdf",
  "filename": "ECTS_Guide_BSWE-pages-1-10.pdf",
  "num_pages": 10
}
```

### ⚡ Fragen stellen zu Dokumenten-Inhalten

In [17]:
question = "Welche LV Art ist Betriebssysteme und wie viele ECTS gibt es da?"
print(f"Question: {question}")
answer = ects_guide_processor.query(question, show_retrieved_chunks=True)
display(Markdown(f"### Answer:\n{answer}"))

Question: Welche LV Art ist Betriebssysteme und wie viele ECTS gibt es da?
Processing query: Welche LV Art ist Betriebssysteme und wie viele ECTS gibt es da?


### Retrieved Document Chunks

**Chunk 1** - Betriebssysteme /  / Operating Systems (Unknown page)


LV Art Course Type, I0859GDI02 = Integrierte Lehrveranstaltung Integrated Course. Semester, I0859GDI02 = 1. Lehreinheiten Teaching units, I0859GDI02 = 60. ECTS, I0859GDI02 = 6 ECTS. Bewertungsmethode Evaluation method, I0859GDI02 = Immanenter Prüfungscharakter Continuous assessment. Lehrveranstaltungsinhalte Content, I0859GDI02 = •  Systematik der Betriebssysteme •  Speichersysteme, Cache und Speicherorganisation •  E/A-Schnittstellen und Kommunikation •  Interrupthandling •  Pipelining •  Superskalare und Multiprozessor-Architekturen •  Sicherheitskonzepte in Betriebssystemen •  Rechteverwaltung •  Unix und Linux •  Grundlagen Maschinencode •  Windows •  Bash und Powershell •  Systematics of operating systems •  Storage systems, cache, and storage organization •  I/O interfaces and communication •  Interrupt handling •  Pipelining •  Superscalar and multiprocessor architectures •  Security concepts in operating systems •  Rights management •  Unix and Linux •  Basics machine code •  Windows •  Bash and PowerShell


---


...

---


**Chunk 5** - Formale Grundlagen und Datenbanken / Formal Foundations and Databases (Unknown page)


LV Art Course Type, I0859GDI03 = Integrierte Lehrveranstaltung Integrated Course. Semester, I0859GDI03 = 2. Lehreinheiten Teaching units, I0859GDI03 = 60
Bewertungsmethode Evaluation method, 6 ECTS = Immanenter Prüfungscharakter Continuous assessment. Lehrveranstaltungsinhalte Content, 6 ECTS = •  Formale Grundlagen •  Logische Grundlagen (Aussagenlogik, Prädikatenlogik,  Beweissysteme, Logische Programmierung) •  Formale Sprachen, Automaten •  Algebraische Grundlagen (Relationen-Algebren,  Vektorräume als Algebren, Mengenlehre) •  Datenbanken •  Relationenmodell •  Normierungen •  Anwendung der algebraischen Grundlagen in einem  relationalen Datenbanksystem mittels Modellierung und  SQL •  Formal basics •  Logical basics (propositional logic, predicate logic, proof  systems, logical programming) •  Formal languages, automata •  Algebraic basics (relations-algebras, vector spaces as  algebras, set theory) •  Databases •  Relation model •  Standardizations •  Application of algebraic fundamentals in a relational  database system using modeling and SQL


---


### Answer:
Die LV Art (Lehrveranstaltungsart) von "Betriebssysteme" ist "Integrierte Lehrveranstaltung", wie im Excerpt 1 angegeben. Es gibt 6 ECTS für diese Lehrveranstaltung, basierend auf der Information im selben Excerpt.

In [ ]:
question = "In welchem Fach gibt es Deskriptive Statistik?"
print(f"Question: {question}")
answer = ects_guide_processor.query(question, show_retrieved_chunks=True)
display(Markdown(f"### Answer:\n{answer}"))

### Dokumenten-Statistik und -Analyse

In [19]:
# Display document information
if ects_guide_processor.document_info:
    display(Markdown(f"### Document Information"))
    display(Markdown(f"**Filename:** {ects_guide_processor.document_info.get('filename', 'Unknown')}"))
    display(Markdown(f"**Pages:** {ects_guide_processor.document_info.get('num_pages', 'Unknown')}"))
    display(Markdown(f"**Chunks:** {len(ects_guide_processor.index)}"))
    
    # Calculate average chunk length
    if ects_guide_processor.index:
        avg_chunk_length = sum(len(chunk['text']) for chunk in ects_guide_processor.index) / len(ects_guide_processor.index)
        display(Markdown(f"**Average chunk length:** {avg_chunk_length:.1f} characters"))
        
        # Find unique headings
        all_headings = set()
        for chunk in ects_guide_processor.index:
            for heading in chunk['headings']:
                # If heading is a dict, extract the text; otherwise, assume it's already a string.
                if isinstance(heading, dict):
                    all_headings.add(heading.get('text', ''))
                else:
                    all_headings.add(heading)
        
        display(Markdown(f"**Number of unique section headings:** {len(all_headings)}"))
        
        # Display content type distribution
        content_types = {}
        for chunk in ects_guide_processor.index:
            content_type = chunk.get('content_type', 'Unknown')
            if content_type not in content_types:
                content_types[content_type] = 0
            content_types[content_type] += 1
        
        display(Markdown(f"**Content type distribution:**"))
        for content_type, count in content_types.items():
            display(Markdown(f"- {content_type}: {count} chunks"))
else:
    print("No document has been processed yet.")

### Document Information

**Filename:** ECTS_Guide_BSWE-pages-1-10.pdf

**Pages:** 10

**Chunks:** 18

**Average chunk length:** 723.8 characters

**Number of unique section headings:** 12

**Content type distribution:**

- None: 18 chunks